In [ ]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
SYMBOLS = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'AVAXUSDT', 'LINKUSDT', 'UNIUSDT', 'DOGEUSDT', 'SHIBUSDT', 'TLMUSDT', 'AXSUSDT']

ROOT = '../../datasets/thesis'

def load_data(filename):
    df = pd.read_csv(filename,names=['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day'], header=0)
    # df['diff'] = df['close'] - df['open']
    # df.loc[df['diff']>=0, 'color'] = 'green'
    # df.loc[df['diff']<0, 'color'] = 'red'
    return df

dfs = {}
alldf = None
for symbol in SYMBOLS:
    # symbol = SYMBOLS[0]
    df = load_data(f"{ROOT}/1h/{symbol}_1h.csv")
    if alldf is None:
        alldf = df
    else:
        alldf = alldf.append(df)
    dfs[symbol] = df
    
df = dfs[SYMBOLS[0]]
df['return'] = df['close'].pct_change()

In [ ]:
df

In [ ]:
df2 = dfs[SYMBOLS[1]]
df2.shape

In [ ]:
alldf.shape

In [ ]:
allchange = alldf.pivot(index="date", columns="tic", values="close").pct_change()
allchange

In [ ]:
unique_date = allchange.index.unique()
time_period = 180
start = time_period
turbulence_index = [0] * start

In [ ]:
count = 0
for i in range(start, len(unique_date)):
    current_price = allchange[allchange.index == unique_date[i]]
    # use one year rolling window to calcualte covariance
    hist_price = allchange[(allchange.index < unique_date[i]) & (allchange.index >= unique_date[i - time_period])]
    # Drop tickers which has number missing values more than the "oldest" ticker
    filtered_hist_price = hist_price.iloc[hist_price.isna().sum().min():].dropna(axis=1)

    cov_temp = filtered_hist_price.cov()
    current_temp = current_price[[x for x in filtered_hist_price]] - np.mean(filtered_hist_price, axis=0)
    # cov_temp = hist_price.cov()
    # current_temp=(current_price - np.mean(hist_price,axis=0))

    temp = current_temp.values.dot(np.linalg.pinv(cov_temp)).dot(current_temp.values.T)
    if temp > 0:
        count += 1
        if count > 2:
            turbulence_temp = temp[0][0]
        else:
            # avoid large outlier because of the calculation just begins
            turbulence_temp = 0
    else:
        turbulence_temp = 0
    turbulence_index.append(turbulence_temp)

In [ ]:
turbulence_index

In [ ]:
a = df.loc[2]

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 8), layout='tight')
ax1.plot(df['date'], df['close'])
ax2.plot(df['date'], turbulence_index)

In [ ]:
plt.plot(df['date'], df['close'])